### Dependencies

In [38]:
import math
import numpy as np
import random
import pandas 
import plotly.express as px
from scipy import linalg
import statistics

### Sampling Functions

In [39]:
# return Gumbel's function value
def gumbel(x):
    return math.pow(math.e, x) * math.pow(math.e, -math.pow(math.e, x))

# return an array tuple of randomly sampled datapoints
def random_points(start, end, n_points):
    x_datapoints = sorted([start+(end-start)*random.random() for _ in range(n_points)])
    y_datapoints = list(map(gumbel, x_datapoints))
    return np.array(x_datapoints), np.array(y_datapoints)

# return an array tuple of equally distanced datapoints
def equal_points(start, end, n_points):
    x_datapoints = [float(_) for _ in range(start, end+1, 1)]
    y_datapoints = list(map(gumbel, x_datapoints))
    return np.array(x_datapoints), np.array(y_datapoints)

# return an array tuple of datapoints given by the Chebyshev points
def cheby_points(start, end, n_points):
    x_datapoints = sorted([start+(end-start)*((-math.cos((x-1)*math.pi/(n_points-1))+1)/2) for x in range(1, n_points+1)])
    y_datapoints = list(map(gumbel, x_datapoints))  
    return np.array(x_datapoints), np.array(y_datapoints)

### Interpolation

##### Polynomial

In [40]:
# return an array of coefficients using Newton-Rhapson's divided differences
def nr_coeffts(x_datapoints, y_datapoints, n_points):
    a = y_datapoints.copy()
    for k in range(1, n_points):
        a[k:n_points] = (a[k:n_points] - a[k-1])/(x_datapoints[k:n_points] - x_datapoints[k-1])
    return a

# return the function value given by the interpolated function via the Newton-Rhapson method
def nr_evalfunct(coefficients, x_datapoints, x, degree):
    value = coefficients[degree]
    for k in range(1, degree+1):
        value = coefficients[degree-k] + (x - x_datapoints[degree-k])*value
    return value

##### RBF

In [41]:
# return multiquadratic rbf basis
def rbf_basis_multiq(x, x_i, sigma):
    return math.sqrt(math.pow((x-x_i), 2)+math.pow(sigma, 2))

# return an array of coefficients using multiquadratic rbf
def rbf_coeffts(x_datapoints, y_datapoints, n_points, sigma):
    rbfmatrx = np.zeros((9,9))
    for j in range(0, n_points):
        rbfmatrx[j][j] = rbf_basis_multiq(x_datapoints[j], x_datapoints[j], sigma)
        for i in range(j+1, n_points):
            rbfmatrx[i][j] = rbf_basis_multiq(x_datapoints[i], x_datapoints[j], sigma)
            rbfmatrx[j][i] = rbfmatrx[i][j]
    return linalg.solve(rbfmatrx, y_datapoints)

# return the function value given by the interpolated function via the multiquadratic rbf
def rbf_evalfunct(x, x_datapoints, y_datapoints, n_points):
    sigma = statistics.stdev(x_datapoints)
    coeffts = rbf_coeffts(x_datapoints, y_datapoints, n_points, sigma)
    value = 0
    for k in range(n_points):
        value += coeffts[k]*rbf_basis_multiq(x, x_datapoints[k], sigma) 
    return value

### Plotting

In [42]:
# plot the gumbel function
def plot_gumbel(step_size, start, end):
   xdata = []
   ydata = []
   for x in range(start*10, (end)*10+1, step_size):
      xdata.append(x/10)
      ydata.append(gumbel(x/10))
   fig = px.scatter(x=xdata, y=ydata)
   fig.update_layout(
      title_text = "Gumbel function",
      font_family="CMU Serif",
      font_size=15,
      title_font_size=25,
      font_color="#0e0f11",
      margin=dict(l=120, r=120, t=120, b=120)
   )
   fig.show()

# plot the newton-rhapsn polynomial interpolation
def plot_poly_interp(_title, x_datapoints, y_datapoints, step_size, start, end, n_points):
   coeffts = nr_coeffts(x_datapoints, y_datapoints, n_points)
   xdata = []
   ydata = []
   for x in range(start*10, (end)*10+1, step_size):
      xdata.append(x/10)
      ydata.append(nr_evalfunct(coeffts, x_datapoints, x/10, n_points-1))
   fig = px.scatter(x=xdata, y=ydata, title=_title)
   fig.update_layout(
      font_family="CMU Serif",
      font_size=15,
      title_font_size=25,
      font_color="#0e0f11",
      margin=dict(l=120, r=120, t=120, b=120)
   )
   fig.show()

# plot the multiquadratic rbf interpolation
def plot_rbf_interp(_title, x_datapoints, y_datapoints, step_size, start, end, n_points):
   xdata = []
   ydata = []
   for x in range(start*10, (end)*10+1, step_size):
      xdata.append(x/10)
      ydata.append(rbf_evalfunct(x/10, x_datapoints, y_datapoints, n_points))
   fig = px.scatter(x=xdata, y=ydata, title=_title)
   fig.update_layout(
      font_family="CMU Serif",
      font_size=15,
      title_font_size=25,
      font_color="#0e0f11",
      margin=dict(l=120, r=120, t=120, b=120)
   )
   fig.show()

### Reconstruction Error

In [43]:
# return the reconstruction error of the newton rhapson interpolation
def poly_recon_error_nr(x_datapoints, y_datapoints, step_size, start, end, n_points):
    coeffts = nr_coeffts(x_datapoints, y_datapoints, n_points)
    error = 0
    for x in range(start*10, (end)*10+1, step_size):
       error += abs(gumbel(x/10) - nr_evalfunct(coeffts, x_datapoints, x/10, n_points-1))
    return error

def rbf_recon_error_mq(x_datapoints, y_datapoints, step_size, start, end, n_points):
    error = 0
    for x in range(start*10, (end)*10+1, step_size):
       error += abs(gumbel(x/10) - rbf_evalfunct(x/10, x_datapoints, y_datapoints, n_points))
    return error

### Getting the Points

In [44]:
start = -5
end = 3
n_points = 9
step_size = 1   

random_x_datapoints, random_y_datapoints = random_points(start, end, n_points)
fig = px.scatter(x=random_x_datapoints, y=random_y_datapoints)
fig.update_layout(
    title_text="9 Randomly-sampled Points",
    font_family="CMU Serif",
    font_size=15,
    title_font_size=25,
    font_color="#0e0f11",
    margin=dict(l=120, r=120, t=120, b=120)
)
fig.show()

equal_x_datapoints, equal_y_datapoints = equal_points(start, end, n_points)
fig = px.scatter(x=equal_x_datapoints, y=equal_y_datapoints)
fig.update_layout(
    title_text="9 Equidistant Points",
    font_family="CMU Serif",
    font_size=15,
    title_font_size=25,
    font_color="#0e0f11",
    margin=dict(l=120, r=120, t=120, b=120)
)
fig.show()

cheby_x_datapoints, cheby_y_datapoints = cheby_points(start, end, n_points)
fig = px.scatter(x=cheby_x_datapoints, y=cheby_y_datapoints)
fig.update_layout(
    title_text="9 Chebyshev Points",
    font_family="CMU Serif",
    font_size=15,
    title_font_size=25,
    font_color="#0e0f11",
    margin=dict(l=120, r=120, t=120, b=120)
)
fig.show()

### Polynomial Basis

##### Computing the Reconstruction Error

In [45]:
print(poly_recon_error_nr(random_x_datapoints, random_y_datapoints, step_size, start, end, n_points))
print(poly_recon_error_nr(equal_x_datapoints, equal_y_datapoints, step_size, start, end, n_points))
print(poly_recon_error_nr(cheby_x_datapoints, cheby_y_datapoints, step_size, start, end, n_points))

153.98451675587225
1.4152350082555476
0.6823219868208231


##### Plotting the Interpolation

In [46]:
plot_gumbel(step_size, start, end)
plot_poly_interp("Randomly-ampled Points", random_x_datapoints, random_y_datapoints, step_size, start, end, n_points)
plot_poly_interp("Equidistant Points", equal_x_datapoints, equal_y_datapoints, step_size, start, end, n_points)
plot_poly_interp("Chebyshev Points", cheby_x_datapoints, cheby_y_datapoints, step_size, start, end, n_points)

### RBF Basis

##### Computing the Reconstruction Error

In [47]:
print(rbf_recon_error_mq(equal_x_datapoints, equal_y_datapoints, step_size, start, end, n_points))
print(rbf_recon_error_mq(cheby_x_datapoints, cheby_y_datapoints, step_size, start, end, n_points))

0.3871421172979905
0.39836607540270524


##### Plotting the Interpolation 

In [48]:
plot_rbf_interp("Equidistant Points", equal_x_datapoints, equal_y_datapoints, step_size, start, end, n_points)
plot_rbf_interp("Chebyshev Points", cheby_x_datapoints, cheby_y_datapoints, step_size, start, end, n_points)